In [1]:
from boto3.session import Session
import boto3
import botocore
import pandas as pd

In [2]:
from loader import load_credential

In [3]:
ACCESS_KEY = load_credential("AWS_ACCESS_KEY_ID","")
SECRET_ACCESS_KEY = load_credential("AWS_SECRET_ACCESS_KEY","")

In [4]:
class S3Manager():
    def __init__(self):
        self.access_key = load_credential("AWS_ACCESS_KEY_ID","")
        self.secret_access_key = load_credential("AWS_SECRET_ACCESS_KEY","")
        self.s3 =  boto3.resource('s3',
                                aws_access_key_id=ACCESS_KEY,
                                aws_secret_access_key=SECRET_ACCESS_KEY)

    def get_file_lists(self, bucket_name, s3_folder):
        lists = []
        if not s3_folder:
            for s3_file in self.s3.Bucket(bucket_name).objects.all():
                lists.append(s3_file.key)
            return lists
        for s3_file in self.s3.Bucket(bucket_name).objects.all():
            if s3_file.key.split('/')[0] == s3_folder:
                lists.append(s3_file.key.split('/')[1])
        return lists
    
    def extract_training_lists(self, csv_file, file_list):
        df = pd.read_csv(csv_file)
        csv_list = []
        new_file_list = []
        for i in range(len(df)):
            csv_list.append(df['origin_url'][i].split('/')[-1])
        for i in range(len(file_list)):
            for j in range(len(csv_list)):
                if(file_list[i] == csv_list[j]):
                    new_file_list.append(file_list[i])
        return new_file_list
    
    def download_s3_files(self, bucket_name, s3_folder, csv_file, local_path):
        # local_path 는 jupyter를 실행한 폴더 기준
        # 폴더 경로는 \\로 끝나야 함 (for Window)
        # TODO : pandas로 csv 읽고 filelist에 있는 것만 남기기
        file_list = self.get_file_lists(bucket_name, s3_folder)
        file_list = self.extract_training_lists(csv_file, file_list)
        for file in file_list[:3]:
            if s3_folder:
                KEY = s3_folder + '/' + file
            else:
                KEY = file
            print(KEY)
            try:
                self.s3.Bucket(bucket_name).download_file(KEY, local_path + file)
                print('downloaded')
            except botocore.exceptions.ClientError as e:
                if e.response['Error']['Code'] == "404":
                    print("The object does not exist.")
                else:
                    print('----')
                    raise

In [5]:
s3manager = S3Manager()

In [6]:
# download from monde-data : BucketNmae --> 크롤링 파일들 박싱한 데이터 저장한 버킷
# s3manager.download_s3_files('버킷이름', ''폴더이름, 'csv파일', '저장경로')

In [7]:
s3manager.download_s3_files('monde-data', 'original-bag-images-dev', './crop_download_test.csv', './data/')

original-bag-images-dev/009KR55482K_1.jpg
downloaded
original-bag-images-dev/DDQFEHN3HDP_1.jpg
downloaded
original-bag-images-dev/TVJHXSJI4JE_1.jpg
downloaded


In [8]:
# download from temp-originalimage : BucketNmae --> hdf5파일들 저장한 버킷
# s3manager.download_s3_files('버킷이름', ''폴더이름, 'csv파일', '저장경로')
# 폴더가 없으므로 비워둠

In [10]:
s3manager.download_s3_files('temp-originalimage', '', './s3upload_download_test.csv', './data/')

hdf5_0_UE0OBJCPG1B_1.jpg
downloaded
hdf5_2_KBX4Y2I1E0F_1.jpg
downloaded
hdf5_3_K6F6ZHLDUHQ_1.jpg
downloaded
